In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels)= mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_dim=28*28),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
train_images= train_images.reshape((60000, 28*28))
test_images= test_images.reshape((10000, 28*28))

train_images= train_images.astype('float32')/255
test_images= test_images.astype('float32')/255


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.save_weights('drive/MyDrive/weights4/')

In [ ]:
initial_weights=model.get_weights()

In [ ]:
initial_weights= model.get_weights()

# Input to Hidden Weights
print("Input to Hidden Weights")
print("───────────────────────")
print(initial_weights[0])

# Bias vector for Hidden Neurons
print("\nBias vector for Hidden Neurons")
print("────────────────────────────")
print(initial_weights[1])

# Hidden to Output Weights
print("\nHidden to Output Weights")
print("────────────────────────")
print(initial_weights[2])

# Bias for output Neuron
print("\nBias for Output Neuron")
print("───────────────────────")
print(initial_weights[3])


Input to Hidden Weights
───────────────────────
[[-0.01049487 -0.04082505 -0.05311067 ... -0.02461446 -0.01769529
  -0.03066614]
 [-0.04644672 -0.01665151  0.06456935 ... -0.04332058 -0.02204521
  -0.00645933]
 [-0.06531794 -0.0657871  -0.06302863 ... -0.04905161 -0.01629963
   0.06038645]
 ...
 [-0.02509523 -0.01623917 -0.00222937 ... -0.04106769  0.06380601
   0.01144135]
 [-0.06308652  0.0148191  -0.05744663 ... -0.05516207 -0.01310738
   0.0342321 ]
 [ 0.02535346 -0.00919089  0.00314657 ...  0.05279838  0.02427322
   0.06737055]]

Bias vector for Hidden Neurons
────────────────────────────
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size= 100)


Epoch 1/5
600/600 [==============================] - 8s 13ms/step - loss: 0.2470 - accuracy: 0.9300
Epoch 2/5
600/600 [==============================] - 5s 9ms/step - loss: 0.0981 - accuracy: 0.9710
Epoch 3/5
600/600 [==============================] - 5s 8ms/step - loss: 0.0650 - accuracy: 0.9801
Epoch 4/5
600/600 [==============================] - 7s 12ms/step - loss: 0.0469 - accuracy: 0.9855
Epoch 5/5
600/600 [==============================] - 5s 8ms/step - loss: 0.0353 - accuracy: 0.9892


In [ ]:
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9799000024795532


In [ ]:
pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.4 MB/s eta 0:00:00


In [ ]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
import numpy as np

In [ ]:
import numpy as np
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 100
epochs= 5
validation_split = 0.1


num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.80,
                                                             begin_step=0,
                                                             end_step=end_step)
}

model2 = keras.Sequential([
    layers.Dense(512, activation='relu', input_dim=28*28),
    layers.Dense(10, activation='softmax')
])

model2.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model2.load_weights('drive/MyDrive/weights4/')


In [ ]:

model2_weights= model2.get_weights()
# Input to Hidden Weights
print("Input to Hidden Weights")
print("───────────────────────")
print(model2_weights[0])

# Bias vector for Hidden Neurons
print("\nBias vector for Hidden Neurons")
print("────────────────────────────")
print(model2_weights[1])

# Hidden to Output Weights
print("\nHidden to Output Weights")
print("────────────────────────")
print(model2_weights[2])

# Bias for output Neuron
print("\nBias for Output Neuron")
print("───────────────────────")
print(model2_weights[3])

Input to Hidden Weights
───────────────────────
[[-0.         -0.         -0.         ... -0.         -0.
  -0.        ]
 [-0.         -0.          0.06456935 ... -0.         -0.
  -0.        ]
 [-0.06531794 -0.0657871  -0.06302863 ... -0.         -0.
   0.06038645]
 ...
 [-0.         -0.         -0.         ... -0.          0.06380601
   0.        ]
 [-0.06308652  0.         -0.05744663 ... -0.         -0.
   0.        ]
 [ 0.         -0.          0.         ...  0.          0.
   0.06737055]]

Bias vector for Hidden Neurons
────────────────────────────
[ 0.00923608  0.06360403  0.06013612 -0.00159987  0.00090803 -0.07706109
 -0.12722225 -0.02115901 -0.0838457   0.00719486  0.07902507  0.01987891
 -0.02188208  0.04181153  0.06763232  0.06041931  0.05791906  0.01864749
 -0.00813717  0.10731354 -0.00165496  0.0489861   0.00894555  0.06649777
  0.0300867   0.05135164 -0.00750342 -0.08424181  0.03854689 -0.02741542
  0.10848355  0.01651354  0.02925604  0.03225194  0.01321194  0.00486653
 

In [ ]:
# `prune_low_magnitude` requires a recompile.
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model2, **pruning_params)

model_for_pruning.compile(optimizer='rmsprop',
                          loss='sparse_categorical_crossentropy',
                          metrics=['accuracy'])

In [ ]:
model_for_pruning.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_  (None, 512)               803330    
 2 (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_dense_  (None, 10)                10252     
 3 (PruneLowMagnitude)                                           
                                                                 
Total params: 813582 (3.10 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 406532 (1.55 MB)
_________________________________________________________________


In [ ]:
import tempfile
logdir = tempfile.mkdtemp()
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

In [ ]:
model_for_pruning.fit(train_images, train_labels, epochs=5, batch_size= 100, callbacks=callbacks)

Epoch 1/5
600/600 [==============================] - 9s 12ms/step - loss: 0.2669 - accuracy: 0.9239
Epoch 2/5
600/600 [==============================] - 7s 12ms/step - loss: 0.1245 - accuracy: 0.9647
Epoch 3/5
600/600 [==============================] - 6s 10ms/step - loss: 0.0894 - accuracy: 0.9753
Epoch 4/5
600/600 [==============================] - 9s 14ms/step - loss: 0.0709 - accuracy: 0.9801
Epoch 5/5
600/600 [==============================] - 6s 10ms/step - loss: 0.0584 - accuracy: 0.9835


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)
print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:  ', model_for_pruning_accuracy)



Baseline test accuracy: 0.9799000024795532
Pruned test accuracy:   0.9775999784469604


In [ ]:
model2_weights= model_for_pruning.get_weights()

In [ ]:
print("Weights for the original model:")
for i, weight in enumerate(initial_weights):
    print(f"Weight {i + 1} - Shape: {weight.shape}")
    print(weight)

# Iterate through and print all weights for the pruned model
print("\nWeights for the pruned model:")
for i, weight in enumerate(model2_weights):
    print(f"Weight {i + 1} - Shape: {weight.shape}")
    print(weight)

Weights for the original model:
Weight 1 - Shape: (784, 512)
[[-0.01049487 -0.04082505 -0.05311067 ... -0.02461446 -0.01769529
  -0.03066614]
 [-0.04644672 -0.01665151  0.06456935 ... -0.04332058 -0.02204521
  -0.00645933]
 [-0.06531794 -0.0657871  -0.06302863 ... -0.04905161 -0.01629963
   0.06038645]
 ...
 [-0.02509523 -0.01623917 -0.00222937 ... -0.04106769  0.06380601
   0.01144135]
 [-0.06308652  0.0148191  -0.05744663 ... -0.05516207 -0.01310738
   0.0342321 ]
 [ 0.02535346 -0.00919089  0.00314657 ...  0.05279838  0.02427322
   0.06737055]]
Weight 2 - Shape: (512,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [ ]:
print("Weights for the original model:")
for i, weight in enumerate(initial_weights):
    print(f"Weight {i + 1} - Shape: {weight.shape}")
    print(weight)

# Iterate through and print all weights for the pruned model
print("\nWeights for the pruned model:")
for i, weight in enumerate(pruned_weights):
    print(f"Weight {i + 1} - Shape: {weight.shape}")
    print(weight)


Weights for the original model:
Weight 1 - Shape: (784, 512)
[[ 0.02700045  0.00013137  0.0456213  ...  0.04088297 -0.05537191
   0.03707785]
 [-0.06099114  0.06337878 -0.05096041 ... -0.03281263  0.02436842
   0.02593797]
 [-0.06537096 -0.04980753  0.04458294 ...  0.05752823 -0.02537625
   0.03703913]
 ...
 [ 0.02991493 -0.04042678 -0.05210917 ...  0.03798266  0.02630352
  -0.02415903]
 [ 0.00094448  0.05576134  0.03025601 ... -0.06366701 -0.03581608
   0.02825262]
 [-0.007358    0.03549149  0.02898478 ...  0.02629033  0.06240831
   0.01139142]]
Weight 2 - Shape: (512,)
[-3.35483328e-02  3.87791777e-03  5.43366112e-02  5.30741327e-02
  7.16805980e-02  2.14986000e-02  4.84295376e-03 -3.65804248e-02
 -5.55107780e-02  6.53344542e-02  2.48677824e-02  1.81697998e-02
 -7.69754797e-02  6.27382696e-02 -7.53372442e-05  4.18999158e-02
 -3.59975323e-02  2.12241430e-05  2.11209734e-03  5.91290481e-02
  6.11435100e-02  8.95328671e-02 -4.85626273e-02 -4.19506952e-02
  6.56544641e-02 -1.31097697e-02